In [45]:
import pandas as pd
import networkx as nx

# Đọc file coauthor
df = pd.read_csv(r"D:\GitHub\Social_Network_Projects\Code\Project_1_coauthor_edges.csv")
df.columns = ["Author1", "Author2", "Weight"]

# Chuyển Weight về int
df["Weight"] = df["Weight"].astype(str).str.extract(r"(\d+)").astype(int)

# Lọc các cạnh có weight >= 2
df_filtered = df[df["Weight"] >= 2].copy()
print("Edges sau lọc:", df_filtered.shape[0])

# Lưu file mới (tùy chọn)
df_filtered.to_csv("coauthor_edges_filtered.csv", index=False, encoding="utf-8")


Edges sau lọc: 5378


In [46]:
# Đọc file
df = pd.read_csv("coauthor_edges_filtered.csv")

# Tạo graph
G = nx.from_pandas_edgelist(df, "Author1", "Author2", edge_attr="Weight")

print("Số nodes:", G.number_of_nodes())
print("Số edges:", G.number_of_edges())

Số nodes: 1030
Số edges: 5378


### Step 2: Chia train-test cho Link Prediction
-   Trích một phần cạnh (ví dụ 20%) làm test set.
-   Xây dựng train graph từ phần còn lại.

In [47]:
from networkx.algorithms import approximation as approx
import random

edges = list(G.edges()) #Lấy tất cả các cạnh (các hợp tác hiện có giữa các tác giả) trong mạng G.
random.seed(42) #Đặt seed ngẫu nhiên để kết quả có thể lặp lại mỗi lần chạy.
test_size = int(0.2 * len(edges)) #Xác định kích thước tập test bằng 20% tổng số cạnh.
test_edges = random.sample(edges, test_size) #Lấy ngẫu nhiên 20% cạnh làm tập test. Đây là những cạnh giả định bị ẩn để kiểm tra khả năng dự đoán sau này.

train_G = G.copy() #Tạo bản sao của mạng để làm tập huấn luyện.
train_G.remove_edges_from(test_edges)#Xóa các cạnh trong test set khỏi tập train, giả lập rằng chúng chưa tồn tại

print("Số cạnh train:", train_G.number_of_edges()) #In ra số cạnh còn lại trong tập huấn luyện.
print("Số cạnh test:", len(test_edges))#In ra số cạnh trong tập test.


Số cạnh train: 4303
Số cạnh test: 1075


### Step 3: Tính các chỉ số dự đoán liên kết (Link Prediction Scores)

-   Một số thuật toán phổ biến:

Common Neighbors (CN): số lượng bạn chung giữa hai node.

Jaccard Coefficient (JC): tỉ lệ bạn chung / bạn hợp nhất.

Adamic-Adar (AA): trọng số cao cho các node hiếm gặp chung.

Preferential Attachment (PA): kết hợp dựa vào degree.

In [52]:
from networkx.algorithms.link_prediction import jaccard_coefficient, adamic_adar_index, preferential_attachment

# Common Neighbors
cn_scores = [(u, v, len(list(nx.common_neighbors(train_G, u, v)))) 
             for u, v in nx.non_edges(train_G)]
# In top 10 kết quả Common Neighbors
print("Common Neighbors (top 10):")
print(sorted(cn_scores, key=lambda x: x[2], reverse=True)[:10])

# Jaccard
jc_scores = list(jaccard_coefficient(train_G))
# In top 10 Jaccard Coefficient
print("\nJaccard Coefficient (top 10):")
print(sorted(jc_scores, key=lambda x: x[2], reverse=True)[:10])

# Adamic-Adar
aa_scores = list(adamic_adar_index(train_G))
# In top 10 Adamic-Adar Index
print("\nAdamic-Adar Index (top 10):")
print(sorted(aa_scores, key=lambda x: x[2], reverse=True)[:10])

# Preferential Attachment
pa_scores = list(preferential_attachment(train_G))
# In top 10 Preferential Attachment
print("\nPreferential Attachment (top 10):")
print(sorted(pa_scores, key=lambda x: x[2], reverse=True)[:10])



Common Neighbors (top 10):
[('Daniel Blankenberg', 'Michael C. Schatz', 63), ('Suzi Aleksander', 'J. Michael Cherry', 61), ('Anton Nekrutenko', 'Jeremy Goecks', 60), ('Daniel Blankenberg', 'Enis Afgan', 59), ('Suzi Aleksander', 'David P. Hill', 59), ('Michael C. Schatz', 'Jeremy Goecks', 57), ('Björn Grüning', 'Jeremy Goecks', 56), ('Pankaj Jaiswal', 'Harold Drabkin', 53), ('J. Michael Cherry', 'Harold Drabkin', 53), ('David P. Hill', 'J. Michael Cherry', 52)]

Jaccard Coefficient (top 10):
[('Sandra Orchard', 'Rizwan Ishtiaq', 1.0), ('Nancy Cartwright', 'Stephan Hartmann', 1.0), ('Gulsum Gudukbay Akbulut', 'Delphine Larivière', 1.0), ('Gulsum Gudukbay Akbulut', 'Simon Gladman', 1.0), ('Gulsum Gudukbay Akbulut', 'Ruben H.P. Vorderman', 1.0), ('Gulsum Gudukbay Akbulut', 'Catherine J. Bromhead', 1.0), ('Gulsum Gudukbay Akbulut', 'Anne Fouilloux', 1.0), ('Gulsum Gudukbay Akbulut', 'Engy Nasr', 1.0), ('Gulsum Gudukbay Akbulut', 'Simon Bray', 1.0), ('Gulsum Gudukbay Akbulut', 'Assunta D DeS

### Step 4: Lấy top dự đoán


In [ ]:
#  Common Neighbors
cn_scores_sorted = sorted(cn_scores, key=lambda x: x[2], reverse=True)
top_pred = cn_scores_sorted[:10]  # Top 10 cặp tác giả dự đoán
print("Top 10 dự đoán hợp tác (Common Neighbors):")
for u, v, score in top_pred:
    print(u, "-", v, ":", score)


Top 10 dự đoán hợp tác (Common Neighbors):
Daniel Blankenberg - Michael C. Schatz : 63
Suzi Aleksander - J. Michael Cherry : 61
Anton Nekrutenko - Jeremy Goecks : 60
Daniel Blankenberg - Enis Afgan : 59
Suzi Aleksander - David P. Hill : 59
Michael C. Schatz - Jeremy Goecks : 57
Björn Grüning - Jeremy Goecks : 56
Pankaj Jaiswal - Harold Drabkin : 53
J. Michael Cherry - Harold Drabkin : 53
David P. Hill - J. Michael Cherry : 52


In [50]:
#  Jaccard Coefficient
jc_scores_sorted = sorted(jc_scores, key=lambda x: x[2], reverse=True)
top_pred = jc_scores_sorted[:10]  # Top 10 cặp tác giả dự đoán
print("Top 10 dự đoán hợp tác (Jaccard Coefficient):")
for u, v, score in top_pred:
    print(u, "-", v, ":", score)


Top 10 dự đoán hợp tác (Jaccard Coefficient):
Sandra Orchard - Rizwan Ishtiaq : 1.0
Nancy Cartwright - Stephan Hartmann : 1.0
Gulsum Gudukbay Akbulut - Delphine Larivière : 1.0
Gulsum Gudukbay Akbulut - Simon Gladman : 1.0
Gulsum Gudukbay Akbulut - Ruben H.P. Vorderman : 1.0
Gulsum Gudukbay Akbulut - Catherine J. Bromhead : 1.0
Gulsum Gudukbay Akbulut - Anne Fouilloux : 1.0
Gulsum Gudukbay Akbulut - Engy Nasr : 1.0
Gulsum Gudukbay Akbulut - Simon Bray : 1.0
Gulsum Gudukbay Akbulut - Assunta D DeSanto : 1.0


In [ ]:
#  Preferential Attachment
pa_scores_sorted = sorted(pa_scores, key=lambda x: x[2], reverse=True)
top_pred = pa_scores_sorted[:10]  # Top 10 cặp tác giả dự đoán
print("Top 10 dự đoán hợp tác (Preferential Attachment):")
for u, v, score in top_pred:
    print(u, "-", v, ":", score)


Top 10 dự đoán hợp tác (Preferential Attachment):
Suzi Aleksander - Anton Nekrutenko : 6474
Suzi Aleksander - Michael C. Schatz : 6391
Daniel Blankenberg - Suzi Aleksander : 6142
Björn Grüning - Suzi Aleksander : 6059
Vésteinn Thórsson - Suzi Aleksander : 6059
Suzi Aleksander - Jeremy Goecks : 5893
Suzi Aleksander - J. Michael Cherry : 5810
Anton Nekrutenko - Harold Drabkin : 5772
Suzi Aleksander - Enis Afgan : 5727
Daniel Blankenberg - Michael C. Schatz : 5698


In [43]:
#  Adamic-Adar
aa_scores_sorted = sorted(aa_scores, key=lambda x: x[2], reverse=True)
top_pred = aa_scores_sorted[:10]  # Top 10 cặp tác giả dự đoán
print("Top 10 dự đoán hợp tác (Adamic-Adar):")
for u, v, score in top_pred:
    print(u, "-", v, ":", score)


Top 10 dự đoán hợp tác (Adamic-Adar):
Anton Nekrutenko - Jeremy Goecks : 34.74749470630821
Daniel Blankenberg - Michael C. Schatz : 34.729421726194516
Michael C. Schatz - Jeremy Goecks : 32.51430009259391
Daniel Blankenberg - Enis Afgan : 31.64766227872723
Björn Grüning - Jeremy Goecks : 31.601968457147667
Suzi Aleksander - J. Michael Cherry : 29.56147242914665
Suzi Aleksander - David P. Hill : 25.678986322416915
J. Michael Cherry - Harold Drabkin : 24.351265798595914
Pankaj Jaiswal - Harold Drabkin : 24.244724013932743
David P. Hill - J. Michael Cherry : 22.49110704696736


### Step 5: Xuất file nodes / edges cho Gephi
Nodes: danh sách tác giả + các centrality 

Edges: toàn bộ cạnh + weight 

In [49]:
# Centrality
deg_c = nx.degree_centrality(G)
bet_c = nx.betweenness_centrality(G)
close_c = nx.closeness_centrality(G)
largest_cc = max(nx.connected_components(G), key=len)
G_sub = G.subgraph(largest_cc)
eig_c_sub = nx.eigenvector_centrality(G_sub, max_iter=5000)
eig_c = {n: eig_c_sub.get(n, 0.0) for n in G.nodes()}

# Nodes
nodes = pd.DataFrame({
    "Id": list(G.nodes()),
    "Degree": [deg_c[n] for n in G.nodes()],
    "Betweenness": [bet_c[n] for n in G.nodes()],
    "Closeness": [close_c[n] for n in G.nodes()],
    "Eigenvector": [eig_c[n] for n in G.nodes()]
})
nodes.to_csv("nodes_1B_gephi.csv", index=False, encoding="utf-8")

# Edges
edges = pd.DataFrame(list(G.edges(data=True)), columns=["Source", "Target", "Weight"])
edges["Weight"] = edges["Weight"].apply(lambda x: x if isinstance(x, (int,float)) else x.get("Weight",1))
edges.to_csv("edges_1B_gephi.csv", index=False, encoding="utf-8")


### Step 6: Trực quan trên Gephi

Import nodes → Nodes table

Import edges → Edges table

Partition theo community (nếu tính được)

Size theo Degree / Eigenvector / Betweenness

Layout ForceAtlas2 hoặc Fruchterman-Reingold